In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import requests
import pandas as pd
import time

from pandas import DataFrame
from StringIO import StringIO
me = "info@jamieniederer.com"
you = "toma.lukas@gmail.com"
USERNAME = "wp12368441-349353"
PASSWORD = "HHgs152.2"
SERVER = 'wp028.webpack.hosteurope.de'
PORT = 587
SUBJECT = unicode("In need of a creative ninja?", "utf-8")

In [2]:
#paths to letter files
html_path = "/Users/lukastoma/Documents/MyRandomFunctions/dev/letter_text_international.html"
plain_path = "/Users/lukastoma/Documents/MyRandomFunctions/dev/letter_text.txt"

In [3]:
class SenderServer():
    """
    Class to take care of connection and parameters to send emails
    """
    def __init__(self, my_email, server, port, username, password):
        self.me = my_email
        self.server = smtplib.SMTP(server, port)
        self.server.ehlo()
        self.server.starttls()
        self.server.ehlo()
        self.server.login(username, password)
        
    def send_mail(self, msg):
        self.server.sendmail(self.me, [msg["To"]], msg.as_string())
        
    def end_connection(self):
        self.server.quit()

class MessageFormatterAndSender:
    def __init__(self, sender_server, html_path = None, plain_path = None):
        self.server = sender_server
        self.multipart = False
        if html_path is None and plain_path is None:
            raise BaseException("supply location of the text")
        
        if html_path is not None:
            #read html from file
            with open(html_path, "rb") as fp:
                self.html = fp.read()
        else:
            self.html = None
            
        if plain_path is not None:
            with open(plain_path, "rb") as fp:
                self.plain = fp.read()
        else:
            self.plain = None
                
        if html_path is not None and plain_path is not None:
            self.multipart = True
                
    def send_formated_message(self, names_to_fill, email, subject):
        """
        created and send emails authomatically
        """
        if self.html is None and self.plain is None:
            raise BaseException("supply letter text")
        elif (self.html is None or self.plain is None) and self.multipart:
            raise BaseException("cannot use multipart if both not supplied")

        if self.plain is not None:
            plain_new = self.plain %names_to_fill
        if self.html is not None:
            html_new = self.html %names_to_fill
            
        if self.multipart:
            #create message
            msg = MIMEMultipart("alternative")
            msg.attach(MIMEText(plain_new, "plain"))
            msg.attach(MIMEText(html_new, "html"))
        elif self.html is not None:
            msg = MIMEText(html_new, "html")
        elif self.plain is not None:
            msg = MIMEText(plain_new, "plain")

        msg['Subject'] = subject
        msg['From'] = me
        msg['To'] = email
        
        self.server.send_mail(msg)
    

In [21]:
df.head()

,name,website,email_address,send,done
0,Achtung!,www.achtung.nl,office@achtung.nl,yes,NaN
1,Blast Radius,www.blastradius.com,amsterdam@blastradius.com,yes,NaN
2,Booreiland,www.booreiland.nl,info@booreiland.nl,yes,NaN
3,Burst Digital,www.burst-digital.com,info@burst-digital.com,yes,NaN
4,Cartelle,www.cartelle.nl,info@cartelle.nl,yes,NaN


In [ ]:
#read from google doc
#r = requests.get('https://docs.google.com/spreadsheets/d/1Lytul8W-hYwhyWoD2Z4_ZqpkV2iNg0aNoNS27waHr2c/export?format=csv&id')
#r = requests.get('https://docs.google.com/spreadsheets/d/1_DegFZwc_gyhdxP4FUgSOqYHqC__2_m4Or93XlXJkbw/export?format=csv&id')
#data = r.content

#df = pd.read_csv(StringIO(data))
df = pd.read_csv("/Users/lukastoma/Downloads/Agencies Worldwide - Tabellenblatt1.csv")
to_send = df.ix[df.send == "yes"]
to_send.fillna({"done": "not done"}, inplace = True)
to_send = to_send.ix[to_send.done == "not done"]
#print to_send
server = SenderServer(me, SERVER, PORT, USERNAME, PASSWORD)
msg_sender = MessageFormatterAndSender(server, html_path, None)
num_to_send = len(to_send.index)
ind = 0
for i in range(num_to_send):
    ind +=1
#     if ind == 49:
#         raise BaseException("Send 49 emails!!!")
    agency = to_send.iloc[i]
    name = agency["name"]
    email = agency["email_address"]
    #email = "toma.lukas@gmail.com"
    print "Sending for %s to %s (%s out of %s)" %(name, email, ind, num_to_send )
    try:
        msg_sender.send_formated_message(name, email, SUBJECT)
        print "    >Sent successfully..."
    except Exception as e:
        print "    >Failed sending: %s" %e
        print "Waiting..."
        time.sleep(4000)
        server = SenderServer(me, SERVER, PORT, USERNAME, PASSWORD)
        msg_sender.send_formated_message(name, email, SUBJECT)
    
server.end_connection()    

Sending for Toormix to info@toormix.com (1 out of 272)
    >Sent successfully...
Sending for Whatalovelyday to info@whatalovelyday.com (2 out of 272)
    >Sent successfully...
Sending for 6S Marketing to sales@6smarketing.com (3 out of 272)
    >Sent successfully...
Sending for 88 Creative to hello@88creative.ca (4 out of 272)
    >Sent successfully...
Sending for 9thCO Inc. to info@9thco.com (5 out of 272)
    >Sent successfully...
Sending for Arcane to begin@arcane.ws (6 out of 272)
    >Sent successfully...
Sending for Art & Science to info@artscience.ca (7 out of 272)
    >Sent successfully...
Sending for BlueBand Digital to info@bluebanddigital.com (8 out of 272)
    >Sent successfully...
Sending for Cordon Media to info@cordonmedia.com (9 out of 272)
    >Sent successfully...
Sending for Critical Mass to newbusiness@criticalmass.com (10 out of 272)
    >Sent successfully...
Sending for Cundari to newbusiness@cundari.com (11 out of 272)
    >Sent successfully...
Sending for DAC Gr

In [29]:
num_to_send

463

In [8]:
df = pd.read_csv("/Users/lukastoma/Downloads/Agencies Worldwide - Tabellenblatt1.csv")
to_send = df.ix[df.send == "yes"]
to_send.fillna({"done": "not done"}, inplace = True)
to_send = to_send.ix[to_send.done == "not done"]

In [9]:
len(to_send.index)

272